# Part 3: OpenCV Tutorials

In this part, we will familiarize ourselves with OpenCV's tools for deep learning by looking at the [dnn/samples] and [tutorials] for the dnn module.

We will take [samples/dnn/edge_detection.py] as a starting point, as this is a bite-sized task that highlights some useful concepts of the dnn module of OpenCV.
Read more about the "_Holistically-Nested Edge Detection_" [here][model]!

---

## Downloading models

First, we will download a model that has been trained with the [Caffe] framework.
OpenCV can also load networks represented in several other formats.
Take a look at the documentation for [cv.dnn.readNet], which we will employ later.

Just for fun, we will utilize the `download_models.py` script from the [dnn/samples] directory.
You can read more about the script in the [README][samples/README], where you see that its primary use case is to download models specified in [dnn/samples/models.yml] by just referring to the model's name.

Unfortunately, our edge detection net is not present in models.yml, so we locate the correct URLs from the [model] repository.

Run the following cells to get started!

[model]: https://github.com/s9xie/hed
[dnn/samples]: https://github.com/opencv/opencv/tree/4.x/samples/dnn
[tutorials]: https://docs.opencv.org/4.x/d2/d58/tutorial_table_of_content_dnn.html
[samples/dnn/edge_detection.py]: https://github.com/opencv/opencv/blob/4.x/samples/dnn/edge_detection.py
[Caffe]: http://caffe.berkeleyvision.org/
[cv.dnn.readNet]: https://docs.opencv.org/4.x/d6/d0f/group__dnn.html#ga4823489a689bf4edfae7447eb807b067
[samples/README]: https://github.com/opencv/opencv/blob/4.x/samples/dnn/README.md
[dnn/samples/models.yml]: https://github.com/opencv/opencv/blob/4.x/samples/dnn/models.yml

In [9]:
# Obtain the 'download_models.py'
samples_url = "https://raw.githubusercontent.com/opencv/opencv/4.x/samples/dnn"
!wget -qO download_models.py "{samples_url}/download_models.py"

from download_models import downloadFile

# Create dicts that we can feed right into 'downloadFile'
caffemodel = {
    "url": "https://vcl.ucsd.edu/hed/hed_pretrained_bsds.caffemodel",
    "filename": "hed_pretrained_bsds.caffemodel",
    "sha": "2c5d7842f25f880eec62fc610b500c5cf2aa351d"
}

prototxt = {
    "url": "https://raw.githubusercontent.com/s9xie/hed/master/examples/hed/deploy.prototxt",
    "filename": "deploy.prototxt",
    "sha": "4f01b87a9cda4912f6751d4ea7acc8866a26b077"
}

# Download the files.
# Downloading the model might take quite a while, be patient.
config = downloadFile(**prototxt, save_dir=".")
model = downloadFile(**caffemodel, save_dir=".")

  Working on deploy.prototxt
  Getting file deploy.prototxt
  expected SHA1: 4f01b87a9cda4912f6751d4ea7acc8866a26b077
  there is already a file with the same name
  actual SHA1:4f01b87a9cda4912f6751d4ea7acc8866a26b077
  hash match - file already exists, skipping
  Working on hed_pretrained_bsds.caffemodel
  Getting file hed_pretrained_bsds.caffemodel
  expected SHA1: 2c5d7842f25f880eec62fc610b500c5cf2aa351d
  there is already a file with the same name
  actual SHA1:2c5d7842f25f880eec62fc610b500c5cf2aa351d
  hash match - file already exists, skipping


## Modifying the network

If you inspect the `deploy.prototxt` file that describes the network, you may see that there are several layers of type `Crop`.
"Crop layers" receive two input blobs and crop the first one to match the spatial dimensions of the second one.

Our model was trained with a crop layer that crops from the center of the image, while the cv.dnn.CropLayer (and also more recent Caffe crop layer) crops from the top-left corner.

In order to prevent shifted results caused by this discrepancy in cropping logic, we're going to replace OpenCV's Crop layer (that makes top-left cropping) by a centric one.

You can read more about custom layers in the [OpenCV tutorial: Custom deep learning layers support](https://docs.opencv.org/4.9.0/dc/db1/tutorial_dnn_custom_layers.html).
There are several other use cases for custom layers, and playing around with it will enhance your understandig of what layers do in a net.

### Challenge: Define the crop area

Look at the image below, and try to work out the crop area, expressed in terms of coordinates `xstart, ystart` and `xend, yend`.
Remember that the values you should use are in terms of the input_shape.
 
![Cropping the inputs](./getMemoryShapes.png)

Modify the cell below, and run it when you have a solution for the cropping.

In [10]:
import cv2 as cv

class CropLayer(object):
    def __init__(self, params, blobs):
        self.xstart = 0
        self.xend = 0
        self.ystart = 0
        self.yend = 0

    # Our layer receives two inputs. We need to crop the first input blob
    # to match a shape of the second one (keeping batch size and number of channels)
    # Returns layer's output shapes depending on input shapes. 
    def getMemoryShapes(self, inputs):
        input_shape  = inputs[0]
        batch_size, num_channels  = input_shape[0], input_shape[1]
        input_height, input_width = input_shape[2], input_shape[3]

        target_shape = inputs[1]
        target_height, target_width = target_shape[2], target_shape[3]

        # TODO: Define the crop area
        self.ystart = 0
        self.xstart = 0
        self.yend = 0
        self.xend = 0

        return [[batch_size, num_channels, target_height, target_width]]

    # Implementation of layer's logic. Compute outputs for given inputs.
    def forward(self, inputs):
        return [inputs[0][:, :, self.ystart:self.yend, self.xstart:self.xend]]

# cv.dnn_registerLayer('Crop', CropLayer)

## Running the net

Now we are ready to try the edge detection model, so we load it with [cv.dnn.readNet].
Take a look at the documentation. What arguments does `readNet` take, and what kind of object does it return?
Examine the code below and look for usages of `net`. Several concepts from this week's lectures should be familiar.
You should also check out the documentation for [cv.dnn.blobFromImage]. What is a "blob"?

We continue with our main processing loop. Make sure to get the camera index right!

[cv.dnn.readNet]: https://docs.opencv.org/4.x/d6/d0f/group__dnn.html#ga4823489a689bf4edfae7447eb807b067
[cv.dnn.blobFromImage]: https://docs.opencv.org/4.x/d6/d0f/group__dnn.html#ga29f34df9376379a603acd8df581ac8d7

Run the cell below in order to test our edge detector.


In [18]:
net = cv.dnn.readNet(model=model, config=config)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_DEFAULT)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)
window_name = 'Holistically-Nested Edge Detection'
cv.namedWindow('Input', cv.WINDOW_NORMAL)
cv.namedWindow(window_name, cv.WINDOW_NORMAL)

cap = cv.VideoCapture(0)
while cv.waitKey(1) < 0:
    hasFrame, frame = cap.read()
    if not hasFrame:
        cv.waitKey()
        break

    cv.imshow('Input', frame)

    inp = cv.dnn.blobFromImage(frame, scalefactor=1.0, size=(500, 500),
                                mean=(104.00698793, 116.66876762, 122.67891434),
                                swapRB=False, crop=False)
    net.setInput(inp)

    out = net.forward()
    out = out[0, 0]
    out = cv.resize(out, (frame.shape[1], frame.shape[0]))
    cv.imshow(window_name, out)

cv.destroyAllWindows()
cap.release()

### Challenge: Revisited

Oh no! We forgot to register our new layer! In the output video, you should notice how the cropping fails,
and we get a padded border along the top and left edges of the output image.

This does also mean that we haven't tested _your_ cropping yet!

Try again after removing the comment before this line in the previous cell.

```py
# cv.dnn_registerLayer('Crop', CropLayer)
```

In [21]:
# You may have to repeat this in order to re-run the program,
# or you may have to restart the ipykernel/notebook `"¯\_(ツ)_/¯ "`
cap.release()

## Next step
Great work!

You can now proceed to [the final step](4-further-work.md).
